# About this notebook
- Deberta-base starter code
- pip wheels is [here](https://www.kaggle.com/yasufuminakama/nbme-pip-wheels)
- Inference notebook is [here](https://www.kaggle.com/yasufuminakama/nbme-deberta-base-baseline-inference)

If this notebook is helpful, feel free to upvote :)

# Library

In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import ast
import time

from utils.utils import (micro_f1, spans_to_binary, span_micro_f1, create_labels_for_scoring,
                        get_char_probs, get_results, get_predictions, get_score, get_logger, seed_everything,
                        get_result)
from utils.load_data import load_data
from utils.dataset import TrainDataset
from utils.dice_loss import DiceLoss
from utils.train import train_fn, valid_fn, get_optimizer_params, get_scheduler
from models import (DebertaBaseLastFourLayer, DebertaBaseLastMLMLastFourLayer,
                    DebertaBaseLastMLM, DebertaBaseVanila)

import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader

import transformers
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transformers.__version__: 4.19.0.dev0
env: TOKENIZERS_PARALLELISM=true


# CFG

In [2]:
class CFG:
    print_freq=100
    num_workers=4
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    apex=True
    
    epochs=15
    batch_size=4 #32
    fc_dropout=0.2
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
    


class CFG_DebertaBaseVanila(CFG):
    path="DebertaBaseVanila"
    model=DebertaBaseVanila

class CFG_DebertaBaseLastFourLayer(CFG):
    path="DebertaBaseLastFourLayer"
    model=DebertaBaseLastFourLayer

class CFG_DebertaBaseLastMLM(CFG):
    path="DebertaBaseLastMLM"
    model=DebertaBaseLastMLM

class CFG_DebertaBaseLastMLMLastFourLayer(CFG):
    path="DebertaBaseLastMLMLastFourLayer"
    model=DebertaBaseLastMLMLastFourLayer
    dropout_num = 4
    
CFGs = [
    CFG_DebertaBaseVanila,
    CFG_DebertaBaseLastFourLayer,
    CFG_DebertaBaseLastMLM,
    CFG_DebertaBaseLastMLMLastFourLayer
]

# Data Loading

In [3]:
# ====================================================
# Data Loading
# ====================================================
train, patient_notes, features = load_data()
train.head()

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history,annotation_length
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...,1
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...,1
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...,1
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...,2
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...,1


In [4]:
first = train.loc[125]
example = {
    "feature_text": first.feature_text,
    "pn_history": first.pn_history,
    "location_list": first.location,
    "annotation_list": first.annotation
}
for key in example.keys():
    print(key)
    print(example[key])
    print("=" * 100)

feature_text
Caffeine-use
pn_history
17 y/o previously healthy male here with heart pounding. States a few months ago, he started having episodes of feeling like his heart is pound/jumping out of his chest. Denies any anxiety, sweating, shaking, or sense of impending doom during these episodes. States he also had shortness of breath with his last episode. Denies any temperature intolerance, jitteriness, chest pain, abdominal pain, GI symptoms, GU symptoms, or easy bruising/bleeding. States he drinks about 3-4 cups coffee/day and several energy drinks throughout the week. Denies any recent stressors in his life.
ROS: negative except for above
PMH, Surg hx, hospitalizations - none
Meds: occasionally take roommate's adderall
NKDA
FH: mom has thyroid problems, dad had heart attack last year
Social: started college 6-7 months ago, denies tobacco use, drinks 3-4 drinks/day on weekends (neg CAGE screening), tried MJ a few months ago. Sexually active with gf, no STD concerns
location_list
['46

# CV split

In [5]:
# ====================================================
# CV split
# ====================================================
if CFG.n_fold == 1:
    train['fold'] = 0
else:
    Fold = GroupKFold(n_splits=CFG.n_fold)
    groups = train['pn_num'].values
    for n, (train_index, val_index) in enumerate(Fold.split(train, train['location'], groups)):
        train.loc[val_index, 'fold'] = int(n)
    train['fold'] = train['fold'].astype(int)
train.groupby('fold').size()

fold
0    2860
1    2860
2    2860
3    2860
4    2860
dtype: int64

# Dataset

In [6]:
# ====================================================
# Define max_len
# ====================================================
for text_col in ['pn_history']:
    pn_history_lengths = []
    tk0 = tqdm(patient_notes[text_col].fillna("").values, total=len(patient_notes))
    for text in tk0:
        length = len(CFGs[0].tokenizer(text, add_special_tokens=False)['input_ids'])
        pn_history_lengths.append(length)
    print(f'{text_col} max(lengths): {max(pn_history_lengths)}')

for text_col in ['feature_text']:
    features_lengths = []
    tk0 = tqdm(features[text_col].fillna("").values, total=len(features))
    for text in tk0:
        length = len(CFGs[0].tokenizer(text, add_special_tokens=False)['input_ids'])
        features_lengths.append(length)
    print(f'{text_col} max(lengths): {max(features_lengths)}')

CFG.max_len = max(pn_history_lengths) + max(features_lengths) + 3 # cls & sep & sep
# CFG.max_len = 466
print(f"max_len: {CFG.max_len}")

max_len: 466


# Helpler functions

In [7]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold, criterion, OUTPUT_DIR):
    
    LOGGER.info(f"========== {OUTPUT_DIR} fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    
    train_loader = DataLoader(TrainDataset(CFG, train_folds),
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(TrainDataset(CFG, valid_folds),
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CFG.model(CFG, config_path=None, pretrained=True)
    torch.save(model.config, CFG.config_path)
    
    model.to(device)

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    
    best_score = -0.1

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(CFG, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(CFG, valid_loader, model, criterion, device)
        predictions = predictions.reshape((len(valid_folds), CFG.max_len))
        
        # scoring
        score, max_th = get_result(valid_folds, predictions, CFG)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}, Th: {max_th:.1f}')
        
        if best_score < score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.path}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR+f"{CFG.path}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
#     valid_folds[[i for i in range(CFG.max_len)]] = predictions
    pd1 = pd.DataFrame(predictions)
    valid_folds = valid_folds.join(pd1)

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [8]:
if __name__ == '__main__':

    def get_result_score(oof_df, CFG):
        
        predictions = oof_df[[i for i in range(CFG.max_len)]].values
        score, max_th = get_result(oof_df, predictions, CFG)
        LOGGER.info(f'Score: {score:<.4f}, Th: {max_th:.1f}')
        return score, max_th
        

#     criterions = {'BCE': nn.BCEWithLogitsLoss(reduction="none"), 'Dice': DiceLoss()}
    criterions = {'BCE': nn.BCEWithLogitsLoss(reduction="none")}
    
    df_scores = pd.DataFrame({'Name': [], 'Score': [], 'Th':[]})
    
    for criterion_name, criterion in criterions.items():
        for CFG in CFGs:
            OUTPUT_DIR = f'../input/{CFG.path}_{criterion_name}/'
            os.makedirs(OUTPUT_DIR, exist_ok=True)
            CFG.config_path = OUTPUT_DIR + 'config.pth'
            LOGGER = get_logger(OUTPUT_DIR + 'train')
            seed_everything(seed=42)

            oof_df = pd.DataFrame()
            for fold in range(CFG.n_fold):
                if fold in CFG.trn_fold:
                    _oof_df = train_loop(train, fold, criterion, OUTPUT_DIR)
                    oof_df = pd.concat([oof_df, _oof_df])
                    LOGGER.info(f"========== fold: {fold} result ==========")
                    score, max_th = get_result_score(_oof_df, CFG)
                    df_score = pd.DataFrame([[f'{CFG.path}_{criterion_name}_fold{fold}', score, max_th]], columns=df_scores.columns)
                    df_scores = pd.concat([df_scores, df_score], ignore_index=True)
                    df_scores.to_csv('../input/scores.csv', index=False)
            oof_df = oof_df.reset_index(drop=True)
            LOGGER.info(f"========== CV ==========")
            score, max_th = get_result_score(oof_df, CFG)
            df_score = pd.DataFrame([[f'{CFG.path}_{criterion_name}', score, max_th]], columns=df_scores.columns)
            df_scores = pd.concat([df_scores, df_score], ignore_index=True)
            df_scores.to_csv('../input/scores.csv', index=False)
            oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        

========== ../input/DebertaBaseLastMLM_BCE/ fold: 0 training ==========
Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.6/site-packages/torch/cuda/__init__.py:125: UserWarning: 
NVIDIA GeForce RTX 3070 with CUDA capability sm_86 is not compatible with the current P

Epoch: [1][0/2860] Elapsed 0m 0s (remain 44m 27s) Loss: 0.7056(0.7056) Grad: 159619.4062  LR: 0.00002000  
Epoch: [1][100/2860] Elapsed 0m 24s (remain 11m 20s) Loss: 0.1400(0.2668) Grad: 66073.7266  LR: 0.00002000  
Epoch: [1][200/2860] Elapsed 0m 47s (remain 10m 33s) Loss: 0.1092(0.1812) Grad: 4833.3818  LR: 0.00002000  
Epoch: [1][300/2860] Elapsed 1m 10s (remain 10m 3s) Loss: 0.0753(0.1486) Grad: 14049.8672  LR: 0.00002000  
Epoch: [1][400/2860] Elapsed 1m 33s (remain 9m 35s) Loss: 0.0825(0.1280) Grad: 43387.6328  LR: 0.00002000  
Epoch: [1][500/2860] Elapsed 1m 56s (remain 9m 10s) Loss: 0.0539(0.1113) Grad: 26501.4238  LR: 0.00001999  
Epoch: [1][600/2860] Elapsed 2m 20s (remain 8m 47s) Loss: 0.0310(0.0997) Grad: 43691.4180  LR: 0.00001999  
Epoch: [1][700/2860] Elapsed 2m 43s (remain 8m 23s) Loss: 0.0163(0.0906) Grad: 39833.2305  LR: 0.00001999  
Epoch: [1][800/2860] Elapsed 3m 6s (remain 7m 59s) Loss: 0.0291(0.0840) Grad: 50525.4609  LR: 0.00001998  
Epoch: [1][900/2860] Elapsed 

Epoch 1 - avg_train_loss: 0.0398  avg_val_loss: 0.0172  time: 737s
Epoch 1 - Score: 0.8223, Th: 0.7
Epoch 1 - Save Best Score: 0.8223 Model


Epoch: [2][0/2860] Elapsed 0m 0s (remain 15m 45s) Loss: 0.0007(0.0007) Grad: 1843.1735  LR: 0.00001978  
Epoch: [2][100/2860] Elapsed 0m 23s (remain 10m 52s) Loss: 0.0256(0.0167) Grad: 63640.4727  LR: 0.00001977  
Epoch: [2][200/2860] Elapsed 0m 47s (remain 10m 25s) Loss: 0.0245(0.0190) Grad: 10791.1260  LR: 0.00001975  
Epoch: [2][300/2860] Elapsed 1m 10s (remain 10m 0s) Loss: 0.0156(0.0198) Grad: 42546.3516  LR: 0.00001973  
Epoch: [2][400/2860] Elapsed 1m 34s (remain 9m 36s) Loss: 0.0058(0.0190) Grad: 25542.5977  LR: 0.00001972  
Epoch: [2][500/2860] Elapsed 1m 57s (remain 9m 13s) Loss: 0.0063(0.0186) Grad: 43732.1602  LR: 0.00001970  
Epoch: [2][600/2860] Elapsed 2m 20s (remain 8m 49s) Loss: 0.0320(0.0185) Grad: 47262.0508  LR: 0.00001968  
Epoch: [2][700/2860] Elapsed 2m 44s (remain 8m 26s) Loss: 0.0245(0.0183) Grad: 62982.9570  LR: 0.00001966  
Epoch: [2][800/2860] Elapsed 3m 7s (remain 8m 2s) Loss: 0.1475(0.0182) Grad: 292471.6562  LR: 0.00001964  
Epoch: [2][900/2860] Elapsed 3

Epoch 2 - avg_train_loss: 0.0169  avg_val_loss: 0.0133  time: 740s
Epoch 2 - Score: 0.7937, Th: 0.5


...

Epoch 13 - avg_train_loss: 0.0015  avg_val_loss: 0.0313  time: 730s
Epoch 13 - Score: 0.8679, Th: 0.5


Epoch: [14][0/2860] Elapsed 0m 0s (remain 19m 39s) Loss: 0.0000(0.0000) Grad: 22.7737  LR: 0.00000086  
Epoch: [14][100/2860] Elapsed 0m 23s (remain 10m 38s) Loss: 0.0156(0.0015) Grad: 20809.4102  LR: 0.00000083  
Epoch: [14][200/2860] Elapsed 0m 46s (remain 10m 12s) Loss: 0.0000(0.0015) Grad: 4.0594  LR: 0.00000081  
Epoch: [14][300/2860] Elapsed 1m 9s (remain 9m 47s) Loss: 0.0000(0.0013) Grad: 20.2979  LR: 0.00000078  
Epoch: [14][400/2860] Elapsed 1m 32s (remain 9m 24s) Loss: 0.0000(0.0013) Grad: 3.0234  LR: 0.00000075  
Epoch: [14][500/2860] Elapsed 1m 55s (remain 9m 2s) Loss: 0.0009(0.0013) Grad: 53748.6523  LR: 0.00000072  
Epoch: [14][600/2860] Elapsed 2m 18s (remain 8m 39s) Loss: 0.0000(0.0012) Grad: 12.9259  LR: 0.00000069  
Epoch: [14][700/2860] Elapsed 2m 41s (remain 8m 16s) Loss: 0.0000(0.0014) Grad: 4.5043  LR: 0.00000067  
Epoch: [14][800/2860] Elapsed 3m 4s (remain 7m 53s) Loss: 0.0000(0.0015) Grad: 6.3702  LR: 0.00000064  
Epoch: [14][900/2860] Elapsed 3m 27s (remain 7m

Epoch 14 - avg_train_loss: 0.0016  avg_val_loss: 0.0313  time: 727s
Epoch 14 - Score: 0.8680, Th: 0.7


Epoch: [15][0/2860] Elapsed 0m 0s (remain 19m 44s) Loss: 0.0000(0.0000) Grad: 10.3647  LR: 0.00000022  
Epoch: [15][100/2860] Elapsed 0m 23s (remain 10m 33s) Loss: 0.0180(0.0023) Grad: 119601.8828  LR: 0.00000020  
Epoch: [15][200/2860] Elapsed 0m 45s (remain 10m 7s) Loss: 0.0000(0.0015) Grad: 1.5572  LR: 0.00000019  
Epoch: [15][300/2860] Elapsed 1m 8s (remain 9m 44s) Loss: 0.0000(0.0015) Grad: 0.9534  LR: 0.00000018  
Epoch: [15][400/2860] Elapsed 1m 31s (remain 9m 20s) Loss: 0.0000(0.0014) Grad: 5.7390  LR: 0.00000016  
Epoch: [15][500/2860] Elapsed 1m 54s (remain 8m 57s) Loss: 0.0002(0.0013) Grad: 9826.6162  LR: 0.00000015  
Epoch: [15][600/2860] Elapsed 2m 16s (remain 8m 34s) Loss: 0.0081(0.0013) Grad: 19829.9434  LR: 0.00000014  
Epoch: [15][700/2860] Elapsed 2m 39s (remain 8m 11s) Loss: 0.0000(0.0013) Grad: 9.3813  LR: 0.00000012  
Epoch: [15][800/2860] Elapsed 3m 2s (remain 7m 48s) Loss: 0.0053(0.0014) Grad: 12722.1807  LR: 0.00000011  
Epoch: [15][900/2860] Elapsed 3m 24s (rem

Epoch 15 - avg_train_loss: 0.0013  avg_val_loss: 0.0312  time: 716s
Epoch 15 - Score: 0.8677, Th: 0.5
========== fold: 4 result ==========
Score: 0.8696, Th: 0.8
========== CV ==========
Score: 0.8654, Th: 0.5
